In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import Image
plt.rcParams.update({'figure.figsize': [8,10]})


## 100種高粱品種識別

In [ ]:
import time
import glob
import os
import math
import cv2
import builtins
import copy
os.environ['TRIDENT_BACKEND'] = 'pytorch'
os.environ['TRIDENT_HOME'] = './trident'

!pip uninstall tridentx -y
!pip install ../input/trident/tridentx-0.7.5-py3-none-any.whl --upgrade

In [ ]:
import trident as T
from trident import *
from trident.models import efficientnet


In [ ]:
import pandas as pd
df=pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv').dropna()
df

In [ ]:
classnames=df.cultivar.unique().tolist()
classnames=list(sorted(classnames))
print(classnames)

In [ ]:
import glob
all_images=glob.glob('../input/sorghum-id-fgvc-9/train_images/*.*g')
print(len(all_images))

images=[]
labels=[]

for index, row in df.iterrows():
    impath='../input/sorghum-id-fgvc-9/train_images/'+row['image']
    if impath in all_images:
        images.append(impath)
        labels.append(classnames.index(row['cultivar']))
        
print(len(images))
print(len(labels))
    

In [ ]:
import cv2
import numpy as np
cl=CLAHE()
def multi_scale_colors(img,spec=None):
    #print(img.shape)
    
    img=cl(img)
    img1=cv2.resize(img.copy(),(240,240))
    
 
    idx=random.choice(list(range(4)))
    #print('idx:',idx,'{0}:{1},{2}:{3}'.format((idx//2)*512,(idx//2+1)*512,(idx%2)*512,(idx%2+1)*512))
    crop_image=img.copy()[(idx//2)*512:(idx//2+1)*512,(idx%2)*512:(idx%2+1)*512,:]
    #print(crop_image.shape)
    img2=cv2.resize(crop_image,(240,240), interpolation=cv2.INTER_AREA)
    
    idxes=random.choices(list(range(16)),k=4)
    #print('idxes:',idxes)
    img3=cv2.resize(img.copy()[(idxes[0]//4)*256:(idxes[0]//4+1)*256,(idxes[0]%4)*256:(idxes[0]%4+1)*256,:],(240,240), interpolation=cv2.INTER_AREA)
    img4=cv2.resize(img.copy()[(idxes[1]//4)*256:(idxes[1]//4+1)*256,(idxes[1]%4)*256:(idxes[1]%4+1)*256,:],(240,240), interpolation=cv2.INTER_AREA)
    
    img5=cv2.cvtColor(cv2.resize(img.copy()[(idxes[2]//4)*256:(idxes[2]//4+1)*256,(idxes[2]%4)*256:(idxes[2]%4+1)*256,:],(240,240), interpolation=cv2.INTER_AREA),cv2.COLOR_RGB2HSV)
    img9=cv2.cvtColor(cv2.resize(img.copy()[(idxes[3]//4)*256:(idxes[3]//4+1)*256,(idxes[3]%4)*256:(idxes[3]%4+1)*256,:],(240,240), interpolation=cv2.INTER_AREA),cv2.COLOR_BGR2YCR_CB)
    
    idxes=random.choices(list(range(64)),k=3)
    img6=cv2.resize(img.copy()[(idxes[0]//8)*128:(idxes[0]//8+1)*128,(idxes[0]%8)*128:(idxes[0]%8+1)*128,:],(240,240), interpolation=cv2.INTER_AREA)
    img7=cv2.resize(img.copy()[(idxes[1]//8)*128:(idxes[1]//8+1)*128,(idxes[1]%8)*128:(idxes[1]%8+1)*128,:],(240,240), interpolation=cv2.INTER_AREA)
    idx=random.choice(list(range(64)))
    img8=cv2.cvtColor(cv2.resize(img.copy()[(idxes[2]//8)*128:(idxes[2]//8+1)*128,(idxes[2]%8)*128:(idxes[2]%8+1)*128,:],(240,240), interpolation=cv2.INTER_AREA),cv2.COLOR_RGB2HSV)
    image_lists=[img1,img2,img3,img4,img5,img6,img7,img8,img9]
    random.shuffle(image_lists)
    new_img=np.concatenate([np.concatenate(image_lists[0:3],axis=1),np.concatenate(image_lists[3:6],axis=1),np.concatenate(image_lists[6:9],axis=1)],axis=0)
    
    return new_img



    
display.display(array2image(multi_scale_colors(cl(image2array('../input/sorghum-id-fgvc-9/train_images/2017-06-01__10-26-27-479.png').astype(np.uint8)))))
    

In [ ]:
ds1=ImageDataset(images,object_type=ObjectType.rgb,symbol='images')
ds2=LabelDataset(labels,object_type=ObjectType.classification_label,symbol='target')
#ds3=ImageDataset(images,object_type=ObjectType.rgb,symbol='images2')

ds2.binding_class_names(class_names=classnames)
print(ds2.class_names)
data_provider=DataProvider(traindata=Iterator(data=ds1,label=ds2,batch_size=4))
#data_provider=DataProvider(traindata=Iterator(data=ds1,label=ZipDataset(ds2,ds3),batch_size=4))
data_provider.paired_transform_funcs=[
    RandomTransform(rotation_range=10, zoom_range=(0.9,1.2), shift_range=0.05, shear_range=0.1, random_flip=0.2,keep_prob=0.3,border_mode='zero'), 
    ]

#    RandomBlur(ksize_range=(3,11),keep_prob=0.7),
data_provider.image_transform_funcs = [
    RandomAdjustGamma(gamma_range=(0.6,1.1)),
    RandomAdjustSaturation(value_range=(0.8, 1.6)),
    RandomAdjustContrast(value_range=(0.8, 1.4)),
    multi_scale_colors,
    AutoLevel(),
    SaltPepperNoise(prob=0.002),  # 椒鹽噪音
    Normalize(127.5, 127.5)]

In [ ]:
def space_to_depth(x:np.ndarray, block_size=3):
    sq_size=block_size*block_size
    
    if len(x.shape)==4 and  x.shape[1]==3:
        new_tensors=[]
        for i in range(x.shape[0]):
            new_tensors.append(space_to_depth(x[i]))
        new_tensors=stack(new_tensors,axis=0)
            
    elif len(x.shape)==3:  
        new_tensors=[]
        if len(x.shape)==3 and x.shape[0]>x.shape[-1]:
            x=x.transpose([2,0,1])
        for i in range(block_size*block_size):
            new_tensors.append(x[:,(i//block_size)*240:(i//block_size+1)*240,(i%block_size)*240:(i%block_size+1)*240])
            
        new_tensors=stack(new_tensors,axis=0)
    return new_tensors
        


arr=multi_scale_colors(cl(image2array('../input/sorghum-id-fgvc-9/train_images/2017-06-01__10-26-27-479.png').astype(np.uint8)))
print(arr.shape)
arr=space_to_depth(to_tensor(image_backend_adaption(arr)), block_size=3)
print(arr.shape)
# arr1=space_to_depth(to_tensor(image_backend_adaption(arr)), block_size=3)
# print(arr0.shape)
display.display(array2image(to_numpy(arr)[0].transpose([1,2,0]).astype(np.uint8)))


In [ ]:
data_provider.preview_images()

In [ ]:
#_images,_labels=data_provider.next()

In [ ]:
import torch
import gc
# torch.cuda.synchronize()
# torch.cuda.empty_cache()
gc.collect()

In [ ]:
class TanhExp(Layer):

    def __init__(self,keep_output=False, name=None):
        super(TanhExp, self).__init__(keep_output=keep_output,name=name)
        self._built = True

    def forward(self, x, **kwargs):

        return clip(x*torch.tanh(torch.exp(x)),-2,2)
    

In [ ]:


    
class SorghumNetV1(Layer):
    def __init__(self, n_classes=100):
        super(SorghumNetV1, self).__init__()
        
        _effb1=efficientnet.EfficientNetB1(pretrained=True,input_shape=(3,240,240),include_top=False)
        _effb1.model.top_conv.trainable=True
        _effb1.model.block7b.trainable=True
        _effb1.model.block7b.dropout_rate=0.2
        _effb1.model.top_conv.activation=TanhExp()

        self.n_classes=n_classes
        self.backbone =_effb1.model
        self.agg=Sequential(
            Reshape((9,1280, 8, 8)),
            Aggregation(mode='mean',axis=1,keepdims=False),
            SeparableConv2d_Block((3,3),depth_multiplier=1,strides=1,auto_pad=True,use_bias=False,activation=TanhExp(),normalization='bn'),
            GlobalAvgPool2d(),
        )

        self.decoder=Dense(n_classes,activation=SoftMax())
     
    def forward(self, x):
        new_x=space_to_depth(x, block_size=3)
        B,N,C,H,W=new_x.shape
        new_x=new_x.reshape((B*N,C,H,W))
    
        return self.decoder(self.agg(self.backbone(new_x)))
    
    
    
class SorghumNetV2(Layer):
    def __init__(self, n_classes=100):
        super(SorghumNetV2, self).__init__()
        
        _effb1=efficientnet.EfficientNetB1(pretrained=True,input_shape=(3,240,240),include_top=False)
        _effb1.model.trainable=True
        _effb1.model.block7b.dropout_rate=0.2
        _effb1.model.top_conv.activation=TanhExp()

        self.n_classes=n_classes
        self.backbone =_effb1.model
        self.agg=Sequential(
            Reshape((9,1280, 8, 8)),
            Aggregation(mode='max',axis=1,keepdims=False),
            SeparableConv2d_Block((3,3),depth_multiplier=1,strides=1,auto_pad=True,use_bias=False,activation=TanhExp(),normalization='bn'),
            ShortCut(
                Identity(),
                Sequential(
                GlobalAvgPool2d(),
                Reshape((1280,1,1)),
                Conv2d((1,1),num_filters=100,use_bias=False,activation=TanhExp()),
                Conv2d((1,1),num_filters=1280,use_bias=False,activation=Sigmoid())
                ),mode='dot'
            ),
            GlobalAvgPool2d(),
        )
        
    
     
        self.decoder=Dense(n_classes,weight_norm='l2')
    
    
    def forward(self, x):
        new_x=space_to_depth(x, block_size=3)
        B,N,C,H,W=new_x.shape
        new_x=new_x.reshape((B*N,C,H,W))
    
      
        return self.decoder(self.agg(self.backbone(new_x)))
    

In [ ]:
# sorghumnet_v1=Model(input_shape=(3,720,720),output=SorghumNetV2(100))
# #sorghumnet_v1.load_model('../input/sorghum-100-identification/Models/sorghumnet_v1_b1.pth')
# #sorghumnet_v1.load_model('./Models/sorghumnet_v1_b1.pth')
# sorghumnet_v1.summary()

自蒸餾，顧名思義自己蒸餾自己，嚴格說起來是指，老師正是幾個批次前、並切換成為評估模式的自己，由於在評估模式中，像是dropout, add_noise等操作將會失效，因此數據會呈現較為乾淨的效果，因此雖然是用相同權重。光是關閉droupout，就讓老師模型相對於同樣權重還原回來的學生模型的效度略勝一籌。也因此我們需要把老師模型銓重凍結、設定為評估模式。

<img src='https://docs.google.com/uc?export=download&id=1hfxk-uZ_m9e5yueAFX9y81gjEa8y2IqG'/>

In [ ]:
sorghumnet_v2_teacher=Model(input_shape=(3,720,720),output=SorghumNetV2(100))
sorghumnet_v2_teacher.load_model('../input/sorghum-100-identification-revised2/Models/sorghumnet_v2_b1.pth.tar')
#sorghumnet_v2_teacher.load_model('./Models/sorghumnet_v2_b1.pth')
sorghumnet_v2_teacher.model.eval()
sorghumnet_v2_teacher.trainable=False

sorghumnet_v2_teacher.model.agg[0].keep_output=True
sorghumnet_v2_teacher.summary()
sorghumnet_v2_teacher.training_context['stop_updates']=10000
sorghumnet_v2_teacher.training_context['training_name']='teacher'
#sorghumnet_v2.output_fn =prepare_output

學生模型則是除了從標籤中學習正確答案之外，

In [ ]:
sorghumnet_v2=Model(input_shape=(3,720,720),output=SorghumNetV2(100))
sorghumnet_v2.load_model('../input/sorghum-100-identification-revised2/Models/sorghumnet_v2_b1.pth.tar')
#sorghumnet_v2.load_model('./Models/sorghumnet_v2_b1.pth')
sorghumnet_v2.trainable=True
sorghumnet_v2.training_context['training_name']='student'
sorghumnet_v2.model.agg[0].keep_output=True
sorghumnet_v2.summary()
#sorghumnet_v2.output_fn =prepare_output

In [ ]:
class ArcMarginProductLoss(Layer):
    def __init__(self, scale=32.0, margin=0.80, easy_margin=False, num_filters= 100,name='ArcMarginProductLoss'):
        super(ArcMarginProductLoss, self).__init__()
        self._name=name
        self.num_filters=num_filters
        self.scale = scale
        self.m = margin
        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)

        # make the function cos(theta+m) monotonic decreasing while theta in [0°,180°]
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin
        self.base_loss=CrossEntropyLoss(reduction='mean')


    def forward(self,output, target,**kwargs):
        # cos(theta)
        try:
            cosine=l2_normalize(output)
            
            # cos(theta + m)
            sine = sqrt(1.0 - pow(cosine, 2))
            phi = cosine * self.cos_m - sine * self.sin_m

            if self.easy_margin:
                phi = where(cosine > 0, phi, cosine)
            else:
                phi = where((cosine - self.th) > 0, phi, cosine - self.mm)

            one_hot = zeros_like(cosine,requires_grad=True)
            one_hot.scatter(1, target.view(-1, 1), 1)

            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
            output = output * self.scale
        except Exception as e:
            print(e)
            PrintException()

        loss = self.base_loss(output, target)
        return loss.mean()

In [ ]:
from sklearn import manifold
from tqdm import  tqdm

 
#將arcface視覺化
def visualize(training_context):
    features=[]
    labels=[]
    preds=[]
    model=training_context['current_model']
    epoch=training_context['current_epoch']
    model.eval()
    print('switch to evaluation.')
#         if epoch==1:
  
#             model.block6e.trainable=True
#             model.block6d.trainable=True
#             model.block6c.trainable=True
#             model.block6b.trainable=True
#         elif epoch==11:
#             model.block6a.trainable=True
#         elif epoch==13:
#             model.block5d.trainable=True
#         elif epoch==15:
#             model.block5c.trainable=True


    NUM_COLORS = 100
    cm = plt.get_cmap('gist_rainbow')
    for i in tqdm(range(50)):
        _images,_labels=data_provider2.next()
        _result=to_numpy(argmax(model(to_tensor(_images)),axis=1))

        _features=model.agg[3].branch2[0].output

        for k in range(len(_images)):
            features.append(to_numpy(l2_normalize(_features[k])))
            labels.append(_labels[k])
            preds.append(_result[k])
    print('features',len(features),'labels',len(labels))
    labels=np.array(labels)
    features=np.array(features)
    preds=np.array(preds)

    print('accuracy:{0:.3%}'.format(np.equal(preds,labels).astype(np.float32).mean()))


    #利用TSNE降維成2維後，繪製成散布圖

    fig = plt.figure(figsize=(12,12))
    ax1= fig.add_subplot(1, 1, 1)
    tsne2 = manifold.TSNE(n_components=2, init='pca', random_state=0)  # 利用t-sne將512特徵向量降維至2
    print('tsne 訓練開始')
    features_tsne2 = tsne2.fit_transform(features) 
    #features_tsne2=l2_normalize(features_tsne2)
    print('tsne 訓練結束')
    for i in range(100):
        x_i = features_tsne2[:,0][labels==i]
        y_i = features_tsne2[:,1][labels==i]
        ax1.scatter(x_i,y_i,s=20,marker='o',c=cm(i//3*3.0/NUM_COLORS))

    model.train()
    plt.legend(classnames, loc = 'upper right')
    plt.title('epoch {0}'.format(epoch))
    make_dir_if_need('results')
    plt.savefig('results/epoch{0}.jpg'.format(epoch), bbox_inches='tight')
    plt.show()


In [ ]:
#visualize(sorghumnet_v2.training_context)

In [ ]:
#sorghumnet_v2.load_model('./Models/sorghumnet_v2_b1.pth')
#sorghumnet_v2.load_model('../input/sorghum-100-identification/Models/sorghumnet_v2_b1.pth.tar')

對比學習的重要精神: 
1. 假設最極端的狀況，每一張圖片都是獨一無二，每個圖片自己是自己的正樣本，全世界的其他圖片都是它的負樣本。
2. 如何解決單一圖片只有一個正樣本的問題。\=\>數據增強 (尤其是空間以及色彩變化)
3. 但是這樣會造成必須讓一張圖片透過數據增強產生多個版本圖片後，再送入模型以產生對應的特徵圖，這樣會耗費數倍的算力。
4. 別忘了我們一開始設計的九宮格，正式透過不同的尺度，不同的色彩座標來產生的，它正是內部的數據增強，所以直接取得內部的特徵圖即可不須額外新樣本送入模型。


<img src='https://docs.google.com/uc?export=download&id=1H9EYFLYu365J58QRkfSNWNGusgAUPuw2'/>

In [ ]:
import torch
import torch.nn.functional as F

#batch=8  8*9*256*256
#對比損失
#在這個模型的設計等於每個案例有9種視角(多尺度、不同色彩系統)，這跟對比學習的數據增強是一樣概念
#所以同一個案例不同視角的特徵應該一致
#不同案例特徵應該要有差異
def contractive_loss(features):
    temperature=0.5
    #把批次數*9作為基礎產生標籤   0,1,2,3...0,1,2,3....  (重複9次)
    labels = torch.cat([torch.arange(features.shape[0]) for i in range(9)], dim=0)
    #如果是同個案例者為1其餘為零
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    #把(B,9,128,8,8)變成(B*9,128,64)然後透過平均變成(B*9,128)
    features=features.reshape((features.size(0)*features.size(1),features.size(3),-1)).mean(-1)
    #標準化
    features = F.normalize(features, dim=1)
    #計算相似性矩陣
    similarity_matrix = torch.matmul(features, features.T)

    
    #產生對角線遮罩(自己對自己必定為1，需要排除)
    mask = torch.eye(labels.shape[0], dtype=torch.bool)
    #排除對角線
    labels = labels[~mask].view(labels.shape[0], -1)
    similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)
    
    # 選取正樣本
    positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)

    # 選取負樣本
    negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)
    #print(positives.shape,negatives.shape)
    #疊合
    logits = torch.cat([positives, negatives], dim=1).reshape(-1)
    labels = torch.cat([ones_like(positives), zeros_like(negatives)], dim=1).long().reshape(-1)
    #labels = torch.zeros(logits.shape[0], dtype=torch.long).to(get_device())

    logits = logits /temperature
    return binary_cross_entropy(logits,labels).mean()
    


_OHEM (online hard example mining，線上困難樣本挖掘)_ 是在建模過程中收集預測錯誤之樣本，並進行重放，讓原本較為稀有的困難樣本得以較高的頻率出現，也增進了模型處理這類困難樣本的能力。

In [ ]:
from trident.reinforcement.utils import Rollout
cxt=get_session()
cxt.rollout=Rollout(only_cpu=True)
cxt.loss_fn=CrossEntropyLoss()



#每個批次把暫存的特徵圖保存至training_context['train_data']
def get_features(training_context):
    cxt=get_session()
    model=training_context['current_model']
    data=training_context['train_data']
    data['features']=model.agg[0].output
    step=training_context['steps']
    if training_context['training_name']=='teacher':
        cxt.features=model.agg[0].output.copy()
        if step>0 and step%100==0:
            model.load_state_dict(load('./Models/sorghumnet_v2_b1.pth').state_dict())
            
        

    

#收集困難樣本
def collect_hard_examples(training_context):
    model=training_context['current_model']
    data=training_context['train_data']
    images=data['images']
    target=data['target']
    features=data['features']
    pred=argmax(exp(data['output']),1)
    
    mask=not_equal(pred,target)

    steps=training_context['steps']
    for i in range(images.size(0)):
        if mask[i].item==True:
            if len(cxt.rollout)<=20 or (len(cxt.rollout)>20 and steps%10!=7):
                cxt.rollout.collect('images',images[i])
                cxt.rollout.collect('target',target[i])
                cxt.rollout.collect('features',features[i])
                #print('hard samples:{0} iou:{1:.3%} landmark_rmse:{2:.3%}'.format(len(cxt.rollout),_iou.item(),_landmark_rmse.item()))

                if (len(cxt.rollout)>0 and len(cxt.rollout)%20)or (len(cxt.rollout)<10)==0:
                    print(yellow_color('hard samples:{0}'.format(len(cxt.rollout))))

                if len(cxt.rollout)>300:
                    cxt.rollout.housekeeping(num_samples=100)
           
                   
            
#將訓練樣本局部抽換成困難樣本
def replace_hard_examples(training_context):
    data=training_context['train_data']
    steps=training_context['steps']
    with torch.no_grad():      
        if len(cxt.rollout)>200 and steps%10==7:
            images=data['images']
            target=data['target']
            features=data['features']
            hards_samples=cxt.rollout.get_samples(int_shape(images)[0],recall_last=False)
            data['images']=stack(hards_samples['images'],0).float().detach()
            data['target']=stack(hards_samples['target'],0).long().detach()
            data['features']=stack(hards_samples['features'],0).long().detach()
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()
    

In [ ]:
def knowledge_distillation_loss(features):
    _dtype=features.dtype
    cxt=get_session()
    student_features=features.float()
    if hasattr(cxt,'features'):
        teacher_features=cxt.features.float()
        loss1=abs(student_features-teacher_features.detach()).mean().to(_dtype)
        return loss1
    else:
        return 0
    

In [ ]:
#sorghumnet_v1.load_model('./Models/sorghumnet_v1_b1.pth')
#sorghumnet_v2.load_model('./Models/sorghumnet_v2_b1.pth')

#sorghumnet_v1.load_model('../input/sorghum-100-identification/Models/sorghumnet_v1_b1.pth.tar')
#sorghumnet_v2.load_model('../input/sorghum-100-identification/Models/sorghumnet_v2_b1.pth.tar')
#sorghumnet_v2.model.load_state_dict(sorghumnet_v1.model.state_dict(),False)

sorghumnet_v2_teacher.with_optimizer(optimizer=DiffGrad,lr=1e-8,betas=(0.9, 0.999))\
    .with_loss(ArcMarginProductLoss(scale=16.0, margin=1.5, easy_margin=True, num_filters=100),as_metric=True)\
    .with_metric(accuracy)\
    .with_metric(accuracy,topk=3,name='top3_accuracy',print_only=True)\
    .with_regularizer('l2',1e-7) \
    .with_model_save_path('./Models/sorghumnet_v2_teacher.pth')\
    .with_accumulate_grads(10)\
    .with_learning_rate_scheduler(CosineLR(min_lr=1e-5,period=1000))\
    .trigger_when(when='on_loss_calculation_start',frequency=1,unit='batch',action=get_features)

#ArcMarginProductLoss 間距加大 縮放變小
#加入contractive_loss對比損失
sorghumnet_v2.with_optimizer(optimizer=DiffGrad,lr=2e-4,betas=(0.9, 0.999),gradient_centralization='all')\
    .with_loss(ArcMarginProductLoss(scale=16.0, margin=1.5, easy_margin=True, num_filters=100),as_metric=True)\
    .with_loss(contractive_loss,loss_weight=0.2,as_metric=True)\
    .with_loss(knowledge_distillation_loss,as_metric=True)\
    .with_metric(accuracy)\
    .with_metric(accuracy,topk=3,name='top3_accuracy',print_only=True)\
    .with_regularizer('l2',1e-7) \
    .with_model_save_path('./Models/sorghumnet_v2_b1.pth')\
    .with_accumulate_grads(10)\
    .with_learning_rate_scheduler(CosineLR(min_lr=1e-5,period=1000))\
    .trigger_when(when='on_loss_calculation_start',frequency=1,unit='batch',action=get_features)\
    .trigger_when(when='on_batch_end',frequency=1,unit='batch',action=collect_hard_examples)\
    .trigger_when(when='on_data_received',frequency=1,unit='batch',action=replace_hard_examples)

In [ ]:

plan=TrainingPlan()\
    .add_training_item(sorghumnet_v2_teacher,name='teacher')\
    .add_training_item(sorghumnet_v2,name='student')\
    .with_data_loader(data_provider)\
    .repeat_epochs(100)\
    .with_batch_size(4)\
    .print_progress_scheduling(20,unit='batch')\
    .out_sample_evaluation_scheduling(frequency=100,unit='batch')\
    .display_loss_metric_curve_scheduling(frequency=100,unit='batch',imshow=True)\
    .save_model_scheduling(20,unit='batch')


plan.start_now()

In [ ]:
#清除gpu快取
import torch
if is_gpu_available():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()
    gc.collect()




In [ ]:
#讀取大會給的範例格式是否一致
f=open('../input/sorghum-id-fgvc-9/sample_submission.csv','r',encoding='utf-8-sig')
rows=f.readlines()
print(rows[:5])

In [ ]:
test_imgs=glob.glob('../input/sorghum-id-fgvc-9/test/*.*g')
print(len(test_imgs))
test_imgs=list(sorted(test_imgs))
submission_dict=OrderedDict()
for im_path in test_imgs:
    folder,filename,ext=split_path(im_path)
    submission_dict[filename+ext]=None
    




ds1_test=ImageDataset(test_imgs,object_type=ObjectType.rgb,symbol='images')
ds2_test=ImageDataset(test_imgs,object_type=ObjectType.image_path,symbol='images_path')


data_provider_test=DataProvider(traindata=Iterator(data=ds1_test,label=ds2_test,batch_size=4))


data_provider_test.image_transform_funcs = [
    multi_scale_colors,
    AutoLevel(),
    Normalize(127.5, 127.5)]

In [ ]:
sorghumnet_v2.eval()
n=0
for i,(test_images,test_images_path) in enumerate(data_provider_test):
    result=argmax(sorghumnet_v2(to_tensor(test_images).to(get_device())),1)
    if is_gpu_available():
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()
    for k in range(len(result)):
        folder,filename,ext=split_path(test_images_path[k])
        if filename+ext not in submission_dict:
            print(filename+ext,' not in submission_dict!')
        else:
            submission_dict[filename+ext]=classnames[result[k].item()]
        
    n+=len(test_images)
    if n%100==0:
        print(n)
    if n>=len(test_imgs) and len([k for k,v in submission_dict.items() if v is None])==0:
        break
